# CRYSTAL convergence helper


In [44]:
from crystal_functions.execute import runcry, runprop
from crystal_functions.file_readwrite import Crystal_output, write_crystal_input, write_properties_input
from crystal_functions.file_readwrite import write_crystal_gui, Crystal_input, cry_combine_density, Crystal_density
from crystal_functions.calculate import cry_ads_energy
from crystal_functions.adsorb import sub_ads_indices
from crystal_functions.convert import cry_gui2pmg
from crystal_functions.utils import view_pmg

# pymatgen imports
from pymatgen.core.structure import Molecule, Structure, Lattice
from pymatgen.core.surface import SlabGenerator, generate_all_slabs
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from pymatgen.analysis.adsorption import AdsorbateSiteFinder
from pymatgen.ext.matproj import MPRester
from pymatgen.io.ase import AseAtomsAdaptor
from pymatgen.core.composition import Composition

from ase.visualize import view

import pandas as pd
import shutil 
import numpy as np
from copy import deepcopy

In [46]:
miller_indices = 'all'
miller_indices = [[1,0,0]] #for testing purpouses
thickness = 10

### Substrate

In [50]:
with MPRester("My-ID") as m:    
    #rutile = m.get_structure_by_material_id("mp-2657")
    mgo = m.get_structure_by_material_id("mp-1265")
    anatase = m.get_structure_by_material_id("mp-390")
    cu = m.get_structure_by_material_id("mp-30")
    iron = m.get_structure_by_material_id("mp-13")
mgo.add_oxidation_state_by_element({"Mg": 2, "O": -2})
anatase.add_oxidation_state_by_element({"Ti": 4, "O": -2})
materials = [mgo,anatase,cu,iron]
materials = [mgo] #for testing purpouses 

Ensure the conventional cell is used

In [51]:
bulks = []
for material in materials:
    bulks.append(SpacegroupAnalyzer(material).get_conventional_standard_structure())

### Adsorbate

In [47]:
h2o = Molecule('HHO',[[0.76,0.00,0.50],[-0.76, 0.00,0.50],[0.0, 0.0, 0.0]])
co = Molecule('CO',[[0.0,0.00,0.50],[0.0, 0.0, 1.43]])
h = Molecule('H',[[0.0,0.00,0.50]])
adsorbates = [h2o,co,h]
adsorbates = [h2o]#for testing purpouses 

### Generate the system

In [48]:
def make_stroichiometric(slab,bulk):
    initial_slab_len = len(slab.atomic_numbers)
    while slab.composition.anonymized_formula !=  bulk.composition.anonymized_formula:
        bottom_site = np.where(slab.cart_coords[:,2] == np.min(slab.cart_coords[:,2],axis=0))[0]
        slab.remove_sites(bottom_site)
        if len(slab.atomic_numbers) < initial_slab_len/2:
            return None
    return(slab)

In [52]:
substrates =[]

if miller_indices == 'all':
    for bulk in bulks:
        slabs = generate_all_slabs(bulk, max_index=1, min_slab_size=15, min_vacuum_size=10.0, 
                                       center_slab=True, symmetrize=True, in_unit_planes=False) 
        #Check if the slabs are stoichiometric
        for slab in slabs:
            if slab.composition.anonymized_formula ==  bulk.composition.anonymized_formula and slab.is_polar() == False:
                substrates.append(slab)
            else:
                stoichiometric_slab = make_stroichiometric(slab,bulk)
                if stoichiometric_slab is not None and stoichiometric_slab.is_polar == False:
                    substrates.append(slab)
                    
elif type(miller_indices) == list:
    for bulk in bulks:
        slabs = []
        for miller_index in miller_indices:
            slab = SlabGenerator(bulk, miller_index, thickness, 10., center_slab=True, max_normal_search=5).get_slab()
            if slab.composition.anonymized_formula ==  bulk.composition.anonymized_formula and slab.is_polar() == False:
                substrates.append(slab)
            else:
                stoichiometric_slab = make_stroichiometric(slab,bulk)
                if stoichiometric_slab is not None and stoichiometric_slab.is_polar == False:
                    substrates.append(slab)
        
for slab in substrates:
    print(slab.miller_index, slab.composition, slab.is_polar())

(1, 0, 0) Mg2+6 O2-6 False


In [8]:
view_pmg(substrates[-2])

<Popen: returncode: None args: ['/Users/brunocamino/miniconda3/envs/test_env...>

In [53]:
sub_composition = []
ads_composition = []
miller_ind = []    
n_layers = []
sub_ads_structures = []
for adsorbate in adsorbates:
    for substrate in substrates:  
        print(substrate.composition, substrate.is_polar())
        adsorption_structures = AdsorbateSiteFinder(substrate).adsorb_both_surfaces(adsorbate,repeat=[1,1,1])  
        for i,adsorption_structure in enumerate(adsorption_structures):
            miller_ind.append(substrate.miller_index)
            n_layers.append(len(np.unique(substrate.cart_coords[:,2])))  
            sub_composition.append(substrate.composition.reduced_formula)
            ads_composition.append(str(adsorbate.composition.hill_formula).replace(" ", ""))
            
            labels1 = []
            labels2 = []
            for i,z in enumerate(adsorption_structure.frac_coords[:,2]):
                if z < 0:
                    labels1.append(i)
                if z > 1:
                    labels2.append(i)

            adsorption_structure.translate_sites(labels1,[0.,0.,1.])
            adsorption_structure.translate_sites(labels2,[0.,0.,-1.])
            
            sub_ads_structures.append(adsorption_structure)

Mg2+6 O2-6 False


In [54]:
len(sub_ads_structures)

4

## Prepare the inputs (SCF)

In [55]:
geom_block = ['Adsorption tests\n','EXTERNAL\n']
bs_block = ['BASISSET\n', 'POB-DZVP\n']
func_block = ['DFT\n', 'B3LYP\n', 'XXLGRID\n', 'ENDDFT\n']
scf_block = [['TOLINTEG\n', '7 7 7 7 14\n'],
             ['SHRINK\n', '12 24\n'],
             ['MAXCYCLE\n', '500\n'],
             'DIIS\n',
             'ENDSCF\n']

## Write the substrate/adsorbate input

## Merge densities

In [56]:
file_names = []
#crystal_input = Crystal_input('data/test/crystal_input.d12')

with open('data/calculations/qsub_input.qsub','r') as file:
    qsub_input = file.readlines()

geom_block = ['Adsorption tests\n','EXTERNAL\n']

input_names = []
for i,structure in enumerate(sub_ads_structures[0:50]):
    input_name = 'data/calculations/'+str(sub_composition[i]).replace(" ", "")+'_'+str(ads_composition[i]).replace(" ", "")+'_' \
               +''.join(str(x) for x in structure.miller_index)+'_'+str(i+1)
    input_names.append(input_name)
    queue = qsub_input.copy()
    queue[1] = '#PBS -N %s\n'%(i+1)
    
    bs_block = ['BASISSET\n', 'POB-DZVP\n']
    scf_block = [['TOLINTEG\n', '7 7 7 7 14\n'],
             ['SHRINK\n', '12 24\n'],
             ['MAXCYCLE\n', '120\n'],
             'DIIS\n',
             'ENDSCF\n']
    
    if 29 in structure.atomic_numbers:
        func_block = ['DFT\n', 'EXCHANGE\n', 'PBE\n', 'CORRELAT\n', 'PBE\n', 'XXLGRID\n', 'ENDDFT\n']
    
    elif 26 in structure.atomic_numbers:
        iron_atoms = ' '.join(str(x) for x in np.where(np.array(structure.atomic_numbers) == 29)[0]+1)
        func_block = ['DFT\n', 'SPIN\n','EXCHANGE\n', 'PBE\n', 'CORRELAT\n', 'PBE\n', 'XXLGRID\n', 'ENDDFT\n']
        scf_block = [['TOLINTEG\n', '7 7 7 7 14\n'],
             ['SHRINK\n', '12 24\n'],
             ['MAXCYCLE\n', '120\n'],
             'DIIS\n',
             'ATOMSPIN\n',
             '%s\n'%len(iron_atoms),
             '%s\n'%iron_atoms,
             'ENDSCF\n']
    else:
        func_block = ['DFT\n', 'B3LYP\n', 'XXLGRID\n', 'ENDDFT\n']
    
    
    file_names.append(input_name)
    #write_crystal_input(input_name,crystal_blocks=[geom_block,bs_block,func_block,scf_block],external_obj=structure)
    write_crystal_input(input_name+'.d12',crystal_blocks=[geom_block,bs_block,func_block,scf_block])
    write_crystal_gui(input_name+'.gui',structure)
    scf_block.insert(-1,'GUESSP\n')
    write_crystal_input(input_name+'_guessp.d12',crystal_blocks=[geom_block,bs_block,func_block,scf_block])
    write_crystal_gui(input_name+'_guessp.gui',structure)
    
    indices = sub_ads_indices(structure)
    #Write the substrate
    
    #sub_inp = deepcopy(crystal_input)
    #sub_inp.add_ghost(indices['adsorbate'])
    ghosts = indices['adsorbate']
    bs_block.extend(['GHOSTS\n','%s\n'%len(ghosts),'%s\n'%' '.join(str(x) for x in ghosts)])
    sub_inp_name = '%s_sub'%input_name
    write_crystal_input(sub_inp_name+'.d12',crystal_blocks=[geom_block,bs_block,func_block,scf_block])
    write_crystal_gui(sub_inp_name+'.gui',structure)
    
    #Write the adsorbate
    #indices = sub_ads_indices(structure)
    #ads_inp = deepcopy(crystal_input)
    #ads_inp.add_ghost(indices['substrate'])
    ghosts = indices['substrate']
    bs_block[-2] = '%s\n'%len(ghosts)
    bs_block[-1] = '%s\n'%' '.join(str(x) for x in ghosts)
    ads_inp_name = '%s_ads'%input_name
    write_crystal_input(ads_inp_name+'.d12',crystal_blocks=[geom_block,bs_block,func_block,scf_block])
    write_crystal_gui(ads_inp_name+'.gui',structure)
    
    queue.append('/rds/general/user/gmallia/home/CRYSTAL17_cx1/v2.2gnu/runcryP %s\n'%input_name[18:])
    queue.append('/rds/general/user/gmallia/home/CRYSTAL17_cx1/v2.2gnu/runcryP %s\n'%sub_inp_name[18:])
    queue.append('/rds/general/user/gmallia/home/CRYSTAL17_cx1/v2.2gnu/runcryP %s\n'%ads_inp_name[18:])
    
    with open('data/calculations/%s.qsub'%(i+1), 'w') as file:
        file.writelines(queue)
    print('qsub %s.qsub'%(i+1))

qsub 1.qsub
qsub 2.qsub
qsub 3.qsub
qsub 4.qsub


In [30]:
import sys
sys.path.insert(1, '../../crystal-code-tools/crystal_functions/crystal_functions/')
from file_readwrite import cry_combine_density, Crystal_input

with open('data/test/qsub_input.qsub','r') as file:
    qsub_input = file.readlines()

queue = qsub_input.copy()
queue[1] = '#PBS -N prop\n'

prop_block = ['RDFMWF\n','END']

for structure in input_names[0:10]:
    density1 = '%s_sub.f98'%structure
    density2 = '%s_ads.f98'%structure
    density3 = '%s.f98'%structure
    write_properties_input('%s_guessp.d3'%structure,prop_block)
    cry_combine_density(density1,density2,density3,new_density='%s_guessp.f98'%structure)
    #runprop('%s_guessp.d3'%structure,'%s_guessp.f98'%structure)
    queue.append('/rds/general/user/gmallia/home/CRYSTAL17_cx1/v2.2gnu/runpropP %s_guessp %s_guessp\n'%(structure[18:], structure[18:]))
    
with open('data/calculations/properties.qsub', 'w') as file:
    file.writelines(queue)


### Write qsub GUESSP

In [40]:
with open('data/test/qsub_input.qsub','r') as file:
    qsub_input = file.readlines()
queue = qsub_input.copy()

for i,structure in enumerate(input_names[0:10]):
    queue = qsub_input.copy()
    queue[1] = '#PBS -N %s_guessp\n'%(i+1)
    queue.append('/rds/general/user/gmallia/home/CRYSTAL17_cx1/v2.2gnu/runcryP %s_guessp %s_guessp\n'%(structure[18:],structure[18:]))
    
    with open('data/calculations/%s_guessp.qsub'%(i+1), 'w') as file:
        file.writelines(queue)
    print('qsub %s_guessp.qsub'%(i+1))

qsub 1_guessp.qsub
qsub 2_guessp.qsub
qsub 3_guessp.qsub
qsub 4_guessp.qsub
qsub 5_guessp.qsub
qsub 6_guessp.qsub
qsub 7_guessp.qsub
qsub 8_guessp.qsub
qsub 9_guessp.qsub
qsub 10_guessp.qsub


### Write GUESSP input

In [36]:
import sys
sys.path.insert(1, '../../crystal-code-tools/crystal_functions/crystal_functions/')
from file_readwrite import cry_combine_density, Crystal_input

for structure in input_names[0:10]:
    crystal_input = Crystal_input('%s.d12'%structure)
    crystal_input.add_guessp()
    write_crystal_input('%s_guessp.d12'%structure,crystal_input=crystal_input)

data/calculations/MgO_H2O_110_1.d12 ['B3LYP\n', 'XXLGRID\n', 'ENDDFT\n', ['TOLINTEG\n', '7 7 7 7 14\n'], ['SHRINK\n', '12 24\n'], 'ENDSCF\n']
data/calculations/MgO_H2O_110_2.d12 ['B3LYP\n', 'XXLGRID\n', 'ENDDFT\n', ['TOLINTEG\n', '7 7 7 7 14\n'], ['SHRINK\n', '12 24\n'], 'ENDSCF\n']
data/calculations/MgO_H2O_110_3.d12 ['B3LYP\n', 'XXLGRID\n', 'ENDDFT\n', ['TOLINTEG\n', '7 7 7 7 14\n'], ['SHRINK\n', '12 24\n'], 'ENDSCF\n']
data/calculations/MgO_H2O_110_4.d12 ['B3LYP\n', 'XXLGRID\n', 'ENDDFT\n', ['TOLINTEG\n', '7 7 7 7 14\n'], ['SHRINK\n', '12 24\n'], 'ENDSCF\n']
data/calculations/MgO_H2O_110_5.d12 ['B3LYP\n', 'XXLGRID\n', 'ENDDFT\n', ['TOLINTEG\n', '7 7 7 7 14\n'], ['SHRINK\n', '12 24\n'], 'ENDSCF\n']
data/calculations/MgO_H2O_110_6.d12 ['B3LYP\n', 'XXLGRID\n', 'ENDDFT\n', ['TOLINTEG\n', '7 7 7 7 14\n'], ['SHRINK\n', '12 24\n'], 'ENDSCF\n']
data/calculations/MgO_H2O_100_7.d12 ['B3LYP\n', 'XXLGRID\n', 'ENDDFT\n', ['TOLINTEG\n', '7 7 7 7 14\n'], ['SHRINK\n', '12 24\n'], 'ENDSCF\n']
data/c

## Display results

In [28]:
sub_cycles = []
ads_cycles = []
sys_cycles = []
guessp_cycles = []
for structure in input_names[0:9]:
    output_sys = Crystal_output('%s.out'%structure)
    if output_sys.converged == True:
        sys_cycles.append(output_sys.get_num_cycles())
    else:
        sys_cycles.append('NC')
    
    output_sub = Crystal_output('%s_sub.out'%structure)
    if output_sub.converged == True:
        sub_cycles.append(output_sub.get_num_cycles())
    else:
        sub_cycles.append('NC')
        
    output_ads = Crystal_output('%s_ads.out'%structure)
    if output_ads.converged == True:
        ads_cycles.append(output_ads.get_num_cycles())
    else:
        ads_cycles.append('NC')
    
    output_guessp = Crystal_output('%s_guessp.out'%structure)
    if output_guessp.converged == True:
        guessp_cycles.append(output_guessp.get_num_cycles())
    else:
        guessp_cycles.append('NC')

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



EXITING: a .out file needs to be specified
Traceback (most recent call last):
  File "/Users/brunocamino/miniconda3/envs/test_env/lib/python3.9/site-packages/crystal_functions/file_readwrite.py", line 131, in __init__
    file = open(output_name, 'r')
FileNotFoundError: [Errno 2] No such file or directory: 'data/calculations/MgO_H2O_110_1_guessp.out'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/brunocamino/miniconda3/envs/test_env/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/f2/8kc7y9697m59bwltxjd42y300000gn/T/ipykernel_26862/296530512.py", line 24, in <module>
    output_guessp = Crystal_output('%s_guessp.out'%structure)
  File "/Users/brunocamino/miniconda3/envs/test_env/lib/python3.9/site-packages/crystal_functions/file_readwrite.py", line 136, in __init__
    sys.exit(1)
SystemExit: 1

During h

TypeError: object of type 'NoneType' has no len()

In [ ]:
df = pd.DataFrame(list(zip(input_names,sys_cycles,sub_cycles,ads_cycles, guessp_cycles)),
                 columns=['Name','Sys cycles','Sub cycles','Ads cycles','Sys guessp cycles'])
df

## Calculate the Adsorbtion energy

In [ ]:
sub_E = []
ads_E = []
guessp_E = []
for structure in input_names[0:9]:
    
    output_sub = Crystal_output('%s_sub.out'%structure)
    if output_sub.converged == True:
        sub_E.append(output_sub.get_final_energy())
    else:
        sub_E.append('NC')
        
    output_ads = Crystal_output('%s_ads.out'%structure)
    if output_ads.converged == True:
        ads_E.append(output_ads.get_final_energy())
    else:
        ads_E.append('NC')
    
    output_guessp = Crystal_output('%s_guessp.out'%structure)
    if output_guessp.converged == True:
        guessp_E.append(output_guessp.get_final_energy())
    else:
        guessp_E.append('NC')
energy_adsorption = (guessp_E - (sub_E+ads_E))/2.


In [ ]:
df = pd.DataFrame(list(zip(input_names,sub_E,ads_E,guessp_E, energy_adsorption)),
                 columns=['Name','Sub E','Ads E','Sys E','Adsorption Energy'])
df

In [ ]:
files = [path+'system.out',path+'system_guessp_p_sum.out']
#,'data/system_guessp_fock0.out']
energy = []
cycles = []
e_ads = []
final_energy = []
for file in files:
    energy.append(Crystal_output(file).final_energy())
    cycles.append(Crystal_output(file).num_cycles())
    e_ads.append(cry_ads_energy(Crystal_output(file).final_energy(),
                                Crystal_output(path+'substrate.out').final_energy(),
                                Crystal_output(path+'adsorbate.out').final_energy()))
    final_energy.append(Crystal_output(file).final_energy())
    

In [ ]:
df = pd.DataFrame(list(zip(files,cycles,e_ads,final_energy)),
                 columns=['Name','Cycles','E ads','Final Energy'])
df

# TEST MgO

In [ ]:
write_crystal_gui('data/test/mgo.gui',mgo)
crystal_input = Crystal_input('data/test/crystal_input.d12')
write_crystal_input('data/test/mgo.d12',crystal_input=crystal_input)

input_name = 'data/test/mgo.d12'
path = 'data/test/'
#System
system_inp = Crystal_input('data/test/crystal_input.d12')
system_inp.opt_to_sp()
bsse_sub_inp_name = path+'system.d12'
write_crystal_input(bsse_sub_inp_name,system_inp)

shutil.copy(input_name[:-4]+'.gui',path+'system.gui')

#Substrate

bsse_sub_inp = Crystal_input('data/test/crystal_input.d12')
bsse_sub_inp.add_ghost([1])
bsse_sub_inp.opt_to_sp()
bsse_sub_inp_name = path+'substrate.d12'
write_crystal_input(bsse_sub_inp_name,bsse_sub_inp)

shutil.copy(input_name[:-4]+'.gui',path+'substrate.gui')

#Adsorbate
bsse_ads_inp = Crystal_input('data/test/crystal_input.d12')
bsse_ads_inp.add_ghost([2])
bsse_sub_inp.opt_to_sp()
bsse_ads_inp_name = path+'adsorbate.d12'
write_crystal_input(bsse_ads_inp_name,bsse_ads_inp)

shutil.copy(input_name[:-4]+'.gui',path+'adsorbate.gui')

In [ ]:
import sys
sys.path.insert(1,'../../crystal-code-tools/crystal_functions/crystal_functions/')
from file_readwrite import Crystal_density, cry_combine_density

density1 = Crystal_density('data/test/substrate.f98')
density2 = Crystal_density('data/test/adsorbate.f98')

cry_combine_density('data/test/substrate.f98','data/test/adsorbate.f98','data/test/mgo.f98','data/test/new_density.f98')